In [2]:
#The objective of this project is to have a tool to take a given date in a given cell, the currency from the same row 
#and return the exchange rate always in reference to the Polish zloty

#The sample Excel used for testing is available in this repository

In [3]:
#Modules
import openpyxl
from datetime import datetime, date, timedelta
from networkdays import networkdays
import requests
import bs4

In [4]:
#Bank Holidays in Poland. Why Poland? That is the country where the end user of this program lives
#Needs to be updated with the bank holidays as the year progresses

bankhlpl = [
    date(year=2021, month=1,day=1),
    date(year=2021, month=1,day=6)
]

In [6]:
#Iterators
i = 2
m = 0

#Empty lists to be used below
listoflinks = []
currencylist = []

#Opening the Excel to be analyzed 
#This part can be improved so the name of the excel is not hard-coded
wb = openpyxl.load_workbook("currency excel.xlsx")

#Main Loop to get in two seperate lists the currencies and the NBP links
#The reson for the two lists is that it makes it easier to troubleshoot errors
while True:
    
    f1 = wb.active

    t1 = f1.cell(i,2).value
    currsymb = f1.cell(i,3).value
    
    if f1.cell(i,2).value == None:
        break

    #Due to undisclosed reasons, the date needs to be the day before hat is present in the cell
    datafinal = t1 - timedelta(1)
    
    #Border conditions such as the beginning of the year and guarantee that no weekends or bank holidays
    #are considered as requested by the customer of this project
    while datafinal.weekday() >= 5 or datetime.date(datafinal) in bankhlpl:
        datafinal = datafinal - timedelta(1)
    
    #Day, month and year to be used below 
    dia = datafinal.day
    mes = datafinal.month
    ano = datafinal.year - 2000
    
    if mes < 10:
        mes = "0"+str(mes)

    if dia < 10:
        dia = "0"+str(dia)

    #The start and end dates
    start_date = date(year=2020, month=12,day=31)
    end_date = datetime.date(datafinal)
    
    #Workday in Poland. Probably there is a better method to get the working days but this one works :)
    workday =networkdays.Networkdays(start_date,end_date,bankhlpl)
    workday = len(workday.networkdays())-1
    
    if workday < 10:
        workday = "00"+str(workday)

    elif workday < 100:
        workday = "0"+str(workday)

    else:
        workday = str(workday)
    
    #The information calculated above is used to build the link for the NBP bank webpage
    #The exchange rate is going to be extracted from this link
    link = f"https://www.nbp.pl/kursy/xml/a{workday}z{ano}{mes}{dia}.xml"
    
    #As of the day when this code is being written, the first two days of January aren't working days.
    #This ain't the best method but it works....
    if end_date in [start_date,bankhlpl[0],date(2021,1,2)]:
        link = f"https://www.nbp.pl/kursy/xml/a255z201231.xml"
        
    listoflinks.append(link)
    
    currencylist.append(currsymb)

    i += 1

while m <= len(listoflinks)-1:
    
    res = requests.get(listoflinks[m])
    soup = bs4.BeautifulSoup(res.text,'lxml')

    #Currency symbols and respective values in reference to the Polish Zloty
    kod = soup.select('kod_waluty')
    kurs = soup.select('kurs_sredni')
        
    kodw = currencylist[m]
    
    for j in range(0,len(kod)):
        
        #A lot of if's in this project
        if kodw in kod[j].text:
            
            f1.cell(m+2,4).value = kurs[j].text
            
    m += 1

In [7]:
#Sving the changes and closing the Excel workbook
wb.save("currency excel.xlsx")
wb.close()